# SMPL-details


### Environment Preparation

In [2]:
# Packages you may use very often.
import torch
import numpy as np
from smplx import SMPL
from smplx.vertex_ids import vertex_ids as VERTEX_IDS
from pytorch3d import transforms  # You may use this package when performing rotation representation transformation.

# Things you don't need to care about. They are just for driving the tutorials.
from lib.logger.look_tool import look_tensor
from lib.utils.path_manager import PathManager
from lib.viewer.wis3d_utils import HWis3D as Wis3D
from lib.skeleton import Skeleton_SMPL24

pm = PathManager()

In [2]:
body_model_smpl = SMPL(
        model_path = pm.inputs / 'body_models' / 'smpl',
        gender     = 'neutral',
    )

## Understand the LBS

## Understand the Joint Regressor

You can get J_regressor from `smplx.SMPL(...).J_regressor`. It's used to regress the joints from the vertices. To be specific, it's a matrix of size `(24, 6890)`, and the joint positions can be got from the linear combination of the relative vertices positions.

In [31]:
J_regressor = body_model_smpl.J_regressor  # (24, 6890)
print(J_regressor.shape)

# Get 'g.t.' of the SMPL's output.
smpl_output = body_model_smpl()
v = smpl_output.vertices.detach()
j = smpl_output.joints[:, :24, :].detach()
print(f'v shape = {v.shape}, j shape = {j.shape}')

# Calculate the joints through J_regressor.
j_by_hand = torch.matmul(J_regressor[None], v)
print(f'j_by_hand shape = {j_by_hand.shape}')

# Check the difference.
delta = j - j_by_hand
_ = look_tensor(delta)

torch.Size([24, 6890])
v shape = torch.Size([1, 6890, 3]), j shape = torch.Size([1, 24, 3])
j_by_hand shape = torch.Size([1, 24, 3])
shape = (1, 24, 3)	dtype = torch.float32	device = cpu	min/max/mean/std = [ -0.000000 -> 0.000000 ] ~ ( 0.000000, 0.000000 )


As you can see, they have tiny difference (for the first 24 joints). But if you can get joints from the standard output object, you are still suggested to get the joints from that.  

> Sometimes, we want the SMPL style 24 joints, but we only have the SMPL-X style parameters. As we have already shown, you can't use SMPL-X parameters to get SMPL's output directly. But you can first use a regressor to get the SMPL style 6890 vertices from SMPL-X style 10475 vertices, and then use the `J_regressor` to get the SMPL style 24 joints from the regressed 6890 vertices. We won't talk about these details here, but you can keep this in mind.

Another thing you may need to know is that, the regressor is sparse, which means that most of the elements are zeros. Now let's get into the details.

In [40]:
# Statistics of the J_regressor.
zero_cnt = (J_regressor == 0).sum().item()
total_cnt = J_regressor.numel()
print(f'Among the total {total_cnt}, there are {zero_cnt} zeros, zero rate: {zero_cnt / total_cnt:.6f}%.')

# Visualize the joints.
active_masks = (J_regressor != 0)  # (24, 6890)
J_regressor_wis3d = Wis3D(
        pm.outputs / 'wis3d',
        'SMPL-J_regressor',
    )
# Add first reference skeleton and mesh.
J_regressor_wis3d.add_motion_verts(verts=v.repeat(25, 1, 1), name=f'vertices', offset=0)
J_regressor_wis3d.add_motion_skel(joints=j.repeat(25, 1, 1), bones=Skeleton_SMPL24.bones, colors=Skeleton_SMPL24.bone_colors, name=f'skeleton', offset=0)

# Visualize each part of the J_regressor.
for i in range(24):
    mask = active_masks[i]
    v_masked = v[0, mask]
    # Visualize the things of interest.
    J_regressor_wis3d.set_scene_id(i+1)
    J_regressor_wis3d.add_point_cloud(vertices=v_masked, name=f'VOI-{i}')  # Vertices of interest used to regress i-th joint.
    J_regressor_wis3d.add_spheres(centers=v_masked, radius=0.01, name=f'VOI-{i}')  # VOI used to regress i-th joint.
    J_regressor_wis3d.add_spheres(centers=j[:1, i], radius=0.02, name=f'joint-{i}')  # i-th joint.

Among the total 165360, there are 165124 zeros, zero rate: 0.998573%.


We now visualize the active elements of the regressor to show exactly which vertices influence the joints regressed. You are supposed to interact with the Wis3D UI to check the things you are interested in.

In [41]:
# Start the server. (Remember to terminate the cell before going on.)
!wis3d --vis_dir {pm.outputs / 'wis3d'} --host 0.0.0.0 --port 19090

Serving on http://0.0.0.0:19090
^C


### Understand the Joints 45

You may notice that, when I mention "joints", I always refer to the first 24. But there are actually 45 joints returned by the SMPL model by default. So what are they?

Actually, the 45 joints can be divided into two parts:

1. The common SMPL joints, which are first regressed from shapped vertices, and then transformed by the pose parameters. They are the first 24 joints.
2. Joints selected (by hand, in advance) from vertices. There are 21 of them by default. You can get the name and the vertices index of them from `smplx.vertex_ids.vertex_ids['smplh']`, I print them below.


In [51]:
print(f'Selected joints from vertices.')
print(f'---------------------------------')
print(f'|  joint_name \t|\tvid\t|')
print(f'---------------------------------')
for k, v in VERTEX_IDS['smplh'].items():
    print(f'|  {k}  \t|\t{v}\t|')
print(f'---------------------------------')

Selected joints from vertices.
---------------------------------
|  joint_name 	|	vid	|
---------------------------------
|  nose  	|	332	|
|  reye  	|	6260	|
|  leye  	|	2800	|
|  rear  	|	4071	|
|  lear  	|	583	|
|  rthumb  	|	6191	|
|  rindex  	|	5782	|
|  rmiddle  	|	5905	|
|  rring  	|	6016	|
|  rpinky  	|	6133	|
|  lthumb  	|	2746	|
|  lindex  	|	2319	|
|  lmiddle  	|	2445	|
|  lring  	|	2556	|
|  lpinky  	|	2673	|
|  LBigToe  	|	3216	|
|  LSmallToe  	|	3226	|
|  LHeel  	|	3387	|
|  RBigToe  	|	6617	|
|  RSmallToe  	|	6624	|
|  RHeel  	|	6787	|
---------------------------------


You can check the implementation at `smplx.SMPL` for more details, SMPL provide some APIs to customize the outputs. They will be useful if you need to change the definition of the output joints (in order, or maybe select more joints from vertices).

### Become Lighter

The sparsity of the regressor tells that, if we only need the joint positions, we don't have to get all the vertices. Recall the inference process of SMPL, we found that we can ignore a lot of calculation while performing the Linear Blend Skinning, LBS.